<a href="https://colab.research.google.com/github/ekshustova/ekshustova/blob/main/Recsys_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Callable, List

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as scs
from numpy.linalg import norm

In [6]:
# Скачиваем архив с данными MovieLens 100k
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip -O ml-100k.zip

# Распаковываем архив в папку data
!unzip -uo ml-100k.zip -d data

# Приводим файл с рейтингами к формату CSV
!awk 'BEGIN { print "user,item,values,timestamp" } { gsub("\\t", ",", $0); print $0 }' data/ml-100k/u.data > data/ml-100k/ratings.csv

# Приводим файл с фильмами к формату CSV
!awk 'BEGIN { print "item,title,genres" } { gsub("\\|", ",", $0); print $0 }' data/ml-100k/u.item > data/ml-100k/movies.csv


--2025-01-18 19:54:43--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  10.8MB/s    in 0.4s    

2025-01-18 19:54:44 (10.8 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: data/ml-100k/
  inflating: data/ml-100k/allbut.pl  
  inflating: data/ml-100k/mku.sh     
  inflating: data/ml-100k/README     
  inflating: data/ml-100k/u.data     
  inflating: data/ml-100k/u.genre    
  inflating: data/ml-100k/u.info     
  inflating: data/ml-100k/u.item     
  inflating: data/ml-100k/u.occupation  
  inflating: data/ml-100k/u.user     
  inflating: data/ml-100k/u1.base    
  inflating: data/ml-100k/u1.test    
  inflating: data/

In [7]:
ratings = pd.read_csv("data/ml-100k/ratings.csv")
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings.head()

,user,item,values,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


Так как функция потерь $L$ выпукла вниз, то необходимо и достаточно, чтобы $\large \frac{\partial L}{\partial q_i} = \frac{\partial L}{\partial p_u} = 0$. Тогда, перепишем в матричном виде:

Предположим, что $Q$ - матрица констант.
$$
\frac{\partial L}{\partial p_u} = (p^T_u Q^t-r_u)\centerdot Q +\lambda p^T_u= 0
$$$$
\Rightarrow p_u^T(Q^TQ+\lambda I)=r_u \centerdot Q
$$
Тогда, $$
p_u^T=r_u \centerdot Q(Q^TQ + \lambda I)^{-1}
$$
Следовательно,
$$
p_u^T=r_u \centerdot Q(Q^TQ + \lambda I)^{-1}
$$$$
q_i^T=r_i \centerdot P(P^TP + \lambda I)^{-1}
$$

_____________________________________________________________________________


In [120]:
import numpy as np
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.linalg import solve

class iALSRecommender:
    def __init__(self, R, num_factors=10, lamb=0.01, iterations=10):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.num_factors = num_factors
        self.lamb = lamb
        self.iterations = iterations


        self.P = np.random.rand(self.num_users, self.num_factors)
        self.Q = np.random.rand(self.num_items, self.num_factors)

    def fit(self):
        """Fit the model using ALS."""
        for iteration in range(self.iterations):
            print(f"Iteration {iteration + 1}/{self.iterations}")

            # Фиксируем Q и решае.
            QTQ = self.Q.T.dot(self.Q)
            lambdaI = eye(self.num_factors) * self.lamb

            for u in range(self.P.shape[0]):
                    self.P[u, :] = solve((QTQ + lambdaI),
                                    (self.R[u, :] @ (self.Q)).flatten())
            # Фиксируем P
            PTP = self.P.T.dot(self.P)
            lambdaI = eye(self.num_factors) * self.lamb

            PTP = self.P.T.dot(self.P)
            lambdaI = scs.eye(PTP.shape[0]) * self.lamb
            for i in range(self.Q.shape[0]):
                    self.Q[i, :] = solve((PTP + lambdaI),
                                    (self.R[:, i].T @ (self.P)).flatten())

    def predict(self):
        """Make predictions by computing the dot product of P and Q."""
        return self.P.dot(self.Q.T)

    def predict_user(self, uid):
        """Make predictions by computing the dot product of P and Q."""
        pred_ratings = self.P[uid] @ self.Q.T
        return np.argsort(pred_ratings)[::-1]




In [149]:
def train_test_split(ratings):
    train_data = []
    test_data = []

    for user, group in ratings.groupby('user'):
        group = group.sort_values('timestamp')
        train_data.append(group.iloc[:-1])
        test_data.append(group.iloc[-1])


    train_data = pd.concat(train_data)
    test_data = pd.DataFrame(test_data)

    return train_data, test_data

In [150]:
train_ratings, test_ratings = train_test_split(ratings)

In [162]:
R_train = coo_matrix((train_ratings['values'], (train_ratings['user'] - 1, train_ratings['item'] - 1))).tocsr()

In [163]:
model = iALSRecommender(R_train, num_factors=10, lamb=0.01, iterations=10)
model.fit()

predictions = model.predict()

hits = 0
for _, row in test_ratings.iterrows():
    user_id = row['user'] - 1
    true_item = row['item'] - 1


    user_predictions = predictions[user_id, :]
    top_10_items = np.argsort(user_predictions)[-10:][::-1]  # Топ-10

    if true_item in top_10_items:
        hits += 1

hit_rate = hits / len(test_ratings)
print(hit_rate)


Iteration 1/10
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10
0.04029692470837752


In [112]:
from scipy.sparse.linalg import svds


In [113]:
ratings

,user,item,values,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
...,...,...,...,...
99995,880,476,3,1997-11-22 05:10:44
99996,716,204,5,1997-11-17 19:39:03
99997,276,1090,1,1997-09-20 22:49:55
99998,13,225,2,1997-12-17 22:52:36


#Dense

In [151]:
import numpy as np
import pandas as pd

class iALSRecommenderDense:
    def __init__(self, ratings, num_factors=128, lamb=0.1, iterations=10):
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.num_factors = num_factors
        self.lamb = lamb
        self.iterations = iterations

        # Инициализация матриц P и Q
        self.P = np.random.rand(self.n_users, self.num_factors)
        self.Q = np.random.rand(self.n_items, self.num_factors)

    def fit(self):
        for iteration in range(self.iterations):
            print(f"Iteration {iteration + 1}/{self.iterations}")

            # Обновление  P
            for i in range(self.n_users):
                idx = np.nonzero(self.ratings[i])[0]
                Qi = self.Q[idx]
                Ri = self.ratings[i, idx]
                Ai = Qi.T @ Qi + self.lamb * np.eye(self.num_factors)
                bi = Qi.T @ Ri
                self.P[i] = np.linalg.solve(Ai, bi)

            # Обновление Q
            for j in range(self.n_items):
                idx = np.nonzero(self.ratings[:, j])[0]
                Pi = self.P[idx]
                Rj = self.ratings[idx, j]
                Aj = Pi.T @ Pi + self.lamb * np.eye(self.num_factors)
                bj = Pi.T @ Rj
                self.Q[j] = np.linalg.solve(Aj, bj)

    def predict(self):
        return self.P @ self.Q.T




In [152]:
num_users = ratings['user'].max()
num_items = ratings['item'].max()
R_train = np.zeros((num_users, num_items))

for _, row in train_ratings.iterrows():
    user_id = row['user'] - 1
    item_id = row['item'] - 1
    R_train[user_id, item_id] = row['values']




In [153]:
model = iALSRecommenderDense(R_train, num_factors=128, lamb=0.1, iterations=10)
model.fit()

Iteration 1/10
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10


In [154]:
predictions = model.predict()

In [155]:
hits = 0
for _, row in test_ratings.iterrows():
    user_id = row['user'] - 1
    true_item = row['item'] - 1

    user_predictions = predictions[user_id, :]
    top_10_items = np.argsort(user_predictions)[-10:][::-1]

    if true_item in top_10_items:
        hits += 1

hit_rate = hits / len(test_ratings)
print(f"Hit Rate: {hit_rate:.4f}")

Hit Rate: 0.0255


SVD

In [ ]:
mtrx_df = ratings.pivot(index = 'user', columns ='item', values = 'values').fillna(0)

In [ ]:
mtrx = mtrx_df.to_numpy()

In [ ]:
U, sigma, Vt = svds(mtrx, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [ ]:
hits = 0
for _, row in test_ratings.iterrows():
    user_id = row['user'] - 1
    true_item = row['item'] - 1

    user_predictions = all_predicted_ratings[user_id, :]
    top_10_items = np.argsort(user_predictions)[-10:][::-1]  # Топ-10

    if true_item in top_10_items:
        hits += 1

hit_rate = hits / len(test_ratings)
print(hit_rate)

0.09225874867444327
